<center><h1>COVID-19 Vaccination Progress</h1></center>
<br>
<center><img src="https://c.files.bbci.co.uk/53A9/production/_115371412_gettyimages-1265248637.jpg" width=600></img></center>  

<br>

<h1 style='background:#26A2AB; border:0; color:black'><center>Introduction</center></h1> 

The data contains the following information:  

* **Country** - this is the country for which the vaccination information is provided;     
* **Country ISO Code** - ISO code for the country;   
* **Date**- date for the data entry; for some of the dates we have only the daily vaccinations, for others, only the (cumulative) total;   
* **Total number of vaccin ations** - this is the absolute number of total immunizations in the country;  
* **Total number of people vaccinated** - a person, depending on the immunization scheme, will receive one or more (typically 2) vaccines; at a certain moment, the number of vaccination might be larger than the number of people;  
* **Total number of people fully vaccinated** - this is the number of people that received the entire set of immunization according to the immunization scheme (typically 2); at a certain moment in time, there might be a certain number of people that received one vaccine and another number (smaller) of people that received all vaccines in the scheme;  
* **Daily vaccinations (raw)** - for a certain data entry, the number of vaccination for that date/country;  
* **Daily vaccinations** - for a certain data entry, the number of vaccination for that date/country;  
* **Total vaccinations per hundred** - ratio (in percent) between vaccination number and total population up to the date in the country;  
* **Total number of people vaccinated per hundred** - ratio (in percent) between population immunized and total population up to the date in the country;  
* **Total number of people fully vaccinated per hundred** - ratio (in percent) between population fully immunized and total population up to the date in the country;   
* **Number of vaccinations per day** - number of daily vaccination for that day and country;   
* **Daily vaccinations per million** -  ratio (in ppm) between vaccination number and total population for the current date in the country;    
* **Vaccines used in the country** - total number of vaccines used in the country (up to date);    
* **Source name** - source of the information (national authority, international organization, local organization etc.);   
* **Source website** - website of the source of information;    


<a id="0"></a>

### Content  

* <a href='#1'>Analysis preparation</a>  
* <a href='#2'>What vaccines are used in each country?</a>    
* <a href='#3'>Which vaccination scheme is used most?</a>    
* <a href='#4'>How many are vaccinated (total and as percent from population)?</a>    
* <a href='#5'>How the vaccination progressed</a>  


### Last updated


In [ ]:
import datetime
import os
import time
with os.scandir("/kaggle/input/covid-world-vaccination-progress") as dir_entries:
    for entry in dir_entries:
        unix_timestamp  = int(entry.stat().st_mtime)
        utc_time = time.gmtime(unix_timestamp)
        print(f"Dataset last time updated: {utc_time.tm_year}-{utc_time.tm_mon}-{utc_time.tm_mday}")
        break
        
ldt = datetime.datetime.now()
print(f"Noteboook last time updated: {ldt.year}-{ldt.month}-{ldt.day}")

<a id="1"></a><h1 style='background:#26A2AB; border:0; color:black'><center>Analysis preparation</center></h1>


We initialize the Python packages we will use for data ingestion, preparation and visualization. We will use mostly Plotly for visualization.
Then we read the data file and aggregate the data on few fields (country, iso_code and vaccines - that is the vaccination scheme used in a certain country).

We will mainly look to:
* What vaccination schemes are used in various countries;  
* Total number of vaccinations and percent of vaccinations;  
* Daily vaccinations and daily vaccinations per million;
* Total people vaccinated and percent of people vaccinated;   

In the dataset is also information about the total number of people completely vaccinated (and percent of them).

We visualize the latest (maximum) values and as well for the variation in time of the above mentioned values.


In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data_df = pd.read_csv("/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv")

In [ ]:
country_vaccine = data_df.groupby(["country", "iso_code", "vaccines"])['total_vaccinations', 
                                                                       'total_vaccinations_per_hundred',
                                                                      'daily_vaccinations',
                                                                      'daily_vaccinations_per_million',
                                                                      'people_vaccinated',
                                                                      'people_vaccinated_per_hundred',
                                                                       'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred'
                                                                      ].max().reset_index()
country_vaccine.columns = ["Country", "iso_code", "Vaccines", "Total vaccinations", "Percent", "Daily vaccinations", 
                           "Daily vaccinations per million", "People vaccinated", "People vaccinated per hundred",
                           'People fully vaccinated', 'People fully vaccinated percent']

<small><a href='#0'>Go to top</a></small>  


<a id="2"></a><h1 style='background:#26A2AB; border:0; color:black'><center>What vaccines are used in each country?</center></h1>

Press **Output** button to see a list of countries where each vaccine is used. 

In [ ]:
vaccines = country_vaccine.Vaccines.unique()
for v in vaccines:
    countries = country_vaccine.loc[country_vaccine.Vaccines==v, 'Country'].values
    print(f"Vaccines: {v}: \nCountries: {list(countries)}\n")

In [ ]:
fig = px.choropleth(locations=country_vaccine['Country'], 
                    locationmode="country names",
                    color=country_vaccine['Vaccines'],
                    title="Countries using each vaccine (different colors for each vaccine)",
                    height = 800
                   )
fig.update_layout({'legend_orientation':'v'})
fig.update_layout({'legend_title':'Vaccine scheme'})
fig.show()

<small><a href='#0'>Go to top</a></small>  

<a id="3"></a><h1 style='background:#26A2AB; border:0; color:black'><center>Which vaccination scheme is used most?</center></h1>

## Overall

Let's look first to the vaccination scheme used overall (not splitted per countries).

In [ ]:
vaccine = data_df.groupby(["vaccines"])['total_vaccinations','total_vaccinations_per_hundred',
                                       'daily_vaccinations','daily_vaccinations_per_million'].max().reset_index()
vaccine.columns = ["Vaccines", "Total vaccinations", "Percent", "Daily vaccinations", 
                           "Daily vaccinations per million"]
def draw_trace_bar_vaccine(data, feature, title, xlab, ylab,color='Blue'):
    data = data.sort_values(feature, ascending=False)
    trace = go.Bar(
            x = data['Vaccines'],
            y = data[feature],
            marker=dict(color=color),
            text=data['Vaccines']
        )
    data = [trace]

    layout = dict(title = title,
              xaxis = dict(title = xlab, showticklabels=True, tickangle=45, 
                           zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                           showline=True, linewidth=2, linecolor='black', mirror=True,
                          tickfont=dict(
                            size=10,
                            color='black'),), 
              yaxis = dict(title = ylab, gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                          showline=True, linewidth=2, linecolor='black', mirror=True),
              plot_bgcolor = 'rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0, 0, 0, 0)',
              hovermode = 'closest',
              height = 800
             )
    fig = dict(data = data, layout = layout)
    iplot(fig, filename='draw_trace')

In [ ]:
draw_trace_bar_vaccine(vaccine, 'Total vaccinations', 'Total per vaccine scheme', 'Vaccine', 'Vaccination total', "darkmagenta" )

Some countries are using a mixed vaccination scheme (they are using more than one vaccine).

The mapping is as following:
* Moderna, Pfizer/BioNTech - USA;  
* CNBG, Sinovac - China;  
* Oxford/AstraZeneca, Pfizer/BioNTech', 'Pfizer/BioNTech - UK;  
* Pfizer/BioNTech - mostly EU;  
* Pfizer/BioNTech, Sinopharm - UAE;  
* Sinovac - Turkey;   
* Covaxin, Covishield - India;  


<small><a href='#0'>Go to top</a></small>  

## Per countries

To see the vaccination scheme distribution per countries, we will use treemap representations. 

We look to the total vaccinations, to daily vaccinations values as well as total people vaccinated.

<font color="red">Note</font>: click on a treemap item to navigate down the tree structure and expand the current branch.

In [ ]:
fig = px.treemap(country_vaccine, path = ['Vaccines', 'Country'], values = 'Total vaccinations',
                title="Total vaccinations per country, grouped by vaccine scheme")
fig.show()

In [ ]:
fig = px.treemap(country_vaccine, path = ['Vaccines', 'Country'], values = 'Daily vaccinations',
                title="Daily vaccinations per country, grouped by vaccine scheme")
fig.show()

In [ ]:
fig = px.treemap(country_vaccine, path = ['Vaccines', 'Country'], values = 'People vaccinated',
                title="People vaccinated per country, grouped by vaccine scheme")
fig.show()

<small><a href='#0'>Go to top</a></small>  


<a id="4"></a><h1 style='background:#26A2AB; border:0; color:black'><center>How many are vaccinated (total and as percent from population)?</center></h1>

Let's look now to the countries statistics, irrespective to the vaccine scheme. We will look to the top of the countries by:

- Total number of vaccinations;  
- Percent of vaccinations from entire population;  
- Daily number of vaccinations;  
- Daily number of vaccination per million population;  
- People vaccinated;  
- Percent of vaccinated people from entire population.

In [ ]:
def draw_trace_bar(data, feature, title, xlab, ylab,color='Blue'):
    data = data.sort_values(feature, ascending=False)
    trace = go.Bar(
            x = data['Country'],
            y = data[feature],
            marker=dict(color=color),
            text=data['Country']
        )
    data = [trace]

    layout = dict(title = title,
              xaxis = dict(title = xlab, showticklabels=True, tickangle=45, 
                           zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                           showline=True, linewidth=2, linecolor='black', mirror=True,
                          tickfont=dict(
                            size=10,
                            color='black'),), 
              yaxis = dict(title = ylab, gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                          showline=True, linewidth=2, linecolor='black', mirror=True),
              plot_bgcolor = 'rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0, 0, 0, 0)',
              hovermode = 'closest'
             )
    fig = dict(data = data, layout = layout)
    iplot(fig, filename='draw_trace')


In [ ]:
draw_trace_bar(country_vaccine, 'Total vaccinations', 'Vaccination total per country', 'Country', 'Vaccination total', "Darkgreen" )

In [ ]:
draw_trace_bar(country_vaccine, 'Percent', 'Vaccination percent per country', 'Country', 'Vaccination percent' )

In [ ]:
draw_trace_bar(country_vaccine, 'Daily vaccinations', 'Daily vaccinations per country', 'Country', 'Daily vaccinations', "red" )

In [ ]:
draw_trace_bar(country_vaccine, 'Daily vaccinations per million', 'Daily vaccinations per million per country', 'Country',\
               'Daily vaccinations per million', "magenta" )

In [ ]:
draw_trace_bar(country_vaccine, 'People vaccinated', 'People vaccinated per country', 'Country',\
               'People vaccinated', "lightblue" )

In [ ]:
draw_trace_bar(country_vaccine, 'People vaccinated per hundred', 'People vaccinated per hundred per country', 'Country',\
               'People vaccinated per hundred', "orange" )

In [ ]:
def plot_custom_scatter(df, x, y, size, color, hover_name, title):
    fig = px.scatter(df, x=x, y=y, size=size, color=color,
               hover_name=hover_name, size_max=80, title = title)
    fig.update_layout({'legend_orientation':'h'})
    fig.update_layout(legend=dict(yanchor="top", y=-0.2))
    fig.update_layout({'legend_title':'Vaccine scheme'})
    fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='grey')
    fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='grey')
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
    fig.show()    

In [ ]:
plot_custom_scatter(country_vaccine, x="Total vaccinations", y="Percent", size="Total vaccinations", color="Vaccines",
           hover_name="Country", title = "Vaccinations (Percent vs. total), grouped per country and vaccines")

In [ ]:
plot_custom_scatter(country_vaccine, x="Total vaccinations", y="Daily vaccinations", size="Total vaccinations", color="Vaccines",
           hover_name="Country", title = "Vaccinations (Total vs. Daily) grouped per country and vaccines")

In [ ]:
plot_custom_scatter(country_vaccine, x="Percent", y="Daily vaccinations per million", size="Total vaccinations", color="Vaccines",
           hover_name="Country", title = "Vaccinations (Daily / million vs. Percent) grouped per country and vaccines")

In [ ]:
trace = go.Choropleth(
            locations = country_vaccine['Country'],
            locationmode='country names',
            z = country_vaccine['Total vaccinations'],
            text = country_vaccine['Country'],
            autocolorscale =False,
            reversescale = True,
            colorscale = 'viridis',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 0.5)
            ),
            colorbar = dict(
                title = 'Total vaccinations',
                tickprefix = '')
        )

data = [trace]
layout = go.Layout(
    title = 'Total vaccinations per country',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot(fig)

In [ ]:
trace = go.Choropleth(
            locations = country_vaccine['Country'],
            locationmode='country names',
            z = country_vaccine['Percent'],
            text = country_vaccine['Country'],
            autocolorscale =False,
            reversescale = True,
            colorscale = 'viridis',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 0.5)
            ),
            colorbar = dict(
                title = 'Percent',
                tickprefix = '')
        )

data = [trace]
layout = go.Layout(
    title = 'Total vaccinations per hundred per country',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot(fig)

In [ ]:
trace = go.Choropleth(
            locations = country_vaccine['Country'],
            locationmode='country names',
            z = country_vaccine['Daily vaccinations'],
            text = country_vaccine['Country'],
            autocolorscale =False,
            reversescale = True,
            colorscale = 'viridis',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 0.5)
            ),
            colorbar = dict(
                title = 'Daily vaccinations',
                tickprefix = '')
        )

data = [trace]
layout = go.Layout(
    title = 'Daily vaccinations per country',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot(fig)

In [ ]:
trace = go.Choropleth(
            locations = country_vaccine['Country'],
            locationmode='country names',
            z = country_vaccine['Daily vaccinations per million'],
            text = country_vaccine['Country'],
            autocolorscale =False,
            reversescale = True,
            colorscale = 'viridis',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 0.5)
            ),
            colorbar = dict(
                title = 'Daily vaccinations per million',
                tickprefix = '')
        )

data = [trace]
layout = go.Layout(
    title = 'Daily vaccinations per million per country',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot(fig)

In [ ]:
trace = go.Choropleth(
            locations = country_vaccine['Country'],
            locationmode='country names',
            z = country_vaccine['People vaccinated'],
            text = country_vaccine['Country'],
            autocolorscale =False,
            reversescale = True,
            colorscale = 'viridis',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 0.5)
            ),
            colorbar = dict(
                title = 'People vaccinated',
                tickprefix = '')
        )

data = [trace]
layout = go.Layout(
    title = 'People vaccinated per country',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot(fig)

In [ ]:
trace = go.Choropleth(
            locations = country_vaccine['Country'],
            locationmode='country names',
            z = country_vaccine['People vaccinated per hundred'],
            text = country_vaccine['Country'],
            autocolorscale =False,
            reversescale = True,
            colorscale = 'viridis',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 0.5)
            ),
            colorbar = dict(
                title = 'People vaccinated per hundred',
                tickprefix = '')
        )

data = [trace]
layout = go.Layout(
    title = 'People vaccinated per hundred per country',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot(fig)

<small><a href='#0'>Go to top</a></small>  


<a id="5"></a><h1 style='background:#26A2AB; border:0; color:black'><center>How the vaccination progressed</center></h1>

Let's look to the way the vaccination progressed.

We will look to the values of total vaccination and daily vaccination.

In [ ]:
country_vaccine_time = data_df[["country", "vaccines", "date", 'total_vaccinations', 
                                'total_vaccinations_per_hundred',  'people_vaccinated','people_vaccinated_per_hundred',
                               'daily_vaccinations','daily_vaccinations_per_million', 
                                'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred'
                               ]].dropna()
country_vaccine_time.columns = ["Country", "Vaccines", "Date", 'Total vaccinations', 'Percent', 'People vaccinated', 'People percent',
                               "Daily vaccinations", "Daily vaccinations per million", 
                                'People fully vaccinated', 'People fully vaccinated percent']

In [ ]:
countries = ['Austria', 'Belgium', 'Bulgaria','Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany',
             'Greece', 'Hungary', 'Ireland', 'Israel', 'Italy', 'Latvia','Lithuania', 'Luxembourg', 'Malta',
             'Netherlands', 'Norway','Poland', 'Portugal', 'Romania', 'Serbia', 'Slovakia', 'Spain', 'Sweden',
             'United Kingdom', 'United States', 'China']

In [ ]:
def plot_time_variation_countries_group(data_df, feature, title, countries):
    data = []
    for country in countries:
        df = data_df.loc[data_df.Country==country]
        trace = go.Scatter(
            x = df['Date'],y = df[feature],
            name=country,
            mode = "markers+lines",
            marker_line_width = 1,
            marker_size = 8,
            marker_symbol = 'circle',
            text=df['Country'])
        data.append(trace)
    layout = dict(title = title,
          xaxis = dict(title = 'Date', showticklabels=True,zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                       showline=True, linewidth=2, linecolor='black', mirror=True,
                       tickfont=dict(size=10,color='darkblue'),), 
          yaxis = dict(title = feature, gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                       showline=True, linewidth=2, linecolor='black', mirror=True, type="log"),
                       plot_bgcolor = 'rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0, 0, 0, 0)',
         hovermode = 'x', 
         height=800
         )
    fig = dict(data=data, layout=layout)
    iplot(fig, filename='all_countries')

In [ ]:
plot_time_variation_countries_group(country_vaccine_time, 'Percent', 'Total vaccination percent evolution (selected countries, log scale)', countries)

In [ ]:
plot_time_variation_countries_group(country_vaccine_time, 'Total vaccinations', 'Total vaccination evolution (selected countries, log scale)', countries)

In [ ]:
plot_time_variation_countries_group(country_vaccine_time, 'People percent', 'People vaccinated percent evolution (selected countries, log scale)', countries)

In [ ]:
plot_time_variation_countries_group(country_vaccine_time, 'People vaccinated', 'People vaccinated evolution (selected countries, log scale)', countries)

In [ ]:
plot_time_variation_countries_group(country_vaccine_time, 'Daily vaccinations', 'Daily vaccinations evolution (selected countries, log scale)', countries)

In [ ]:
plot_time_variation_countries_group(country_vaccine_time, 'Daily vaccinations per million', 'Daily vaccinations per million evolution (selected countries, log scale)', countries)

In [ ]:
plot_time_variation_countries_group(country_vaccine_time, 'People fully vaccinated percent', 'People fully vaccinated percent evolution (selected countries, log scale)', countries)

In [ ]:
plot_time_variation_countries_group(country_vaccine_time, 'People fully vaccinated', 'People fully vaccinated evolution (selected countries, log scale)', countries)

<small><a href='#0'>Go to top</a></small>  